# Creating hierarchical brain atlas in BIDS spec
This notebook achieves the following:
- Converts the original hierarchical brain atlas in both MNI152NLin6ASym and MNI152NLin2009cASym space.
    - We will transform the atlas from original MNI152NLin6ASym space to MNI152NLin2009cASym space. 
- Creates both 1mm and 2mm voxel space versions of the atlas.
- Ensures that the naming of the files is consistent with BIDS specifications for atlas files.
    - Create relevant sidecar files for each atlas.

In [1]:
from pathlib import Path
import json
import requests

import ants
import nibabel as nib
import numpy as np
import pandas as pd

atlases = [
    "./data/AA424.nii.gz",
    "./data/AA424+2mm.nii.gz",
    "./data/AA424+4mm.nii.gz",
    "./data/AA424+n90merged.nii.gz",
]

## Create versions of AA424 atlas with fewer number of regions
We will create the following:
- AA7
- AA24
- AA50

In [2]:
df = pd.read_csv("./data/AA_all_maps.csv")
columns = ['A424', 'AA-7', 'AA-24', 'AA-50']
df = df[columns]

df.head()

,A424,AA-7,AA-24,AA-50
0,1,1,22,22
1,2,1,15,15
2,3,1,1,1
3,4,1,22,22
4,5,1,22,22


In [ ]:
for atlas in atlases:
    print(f"Processing atlas: {atlas}")

    # Load the atlas
    atlas_img = nib.load(atlas)
    atlas_data = atlas_img.get_fdata()
    atlas_header = atlas_img.header
    atlas_affine = atlas_img.affine

    atlas_name = atlas.split("/")[-1].replace(".nii.gz", "")
    if atlas_name.startswith("AA424+"):
        atlas_name, suffix = atlas_name.split("+")
        suffix = f"+{suffix}"
    else:
        suffix = ""

    for new_atlas in ['AA-7', 'AA-24', 'AA-50']:
        print(f"Processing column: {new_atlas}")

        # Using df as lookup table, replace values in atlas_data
        new_atlas_data = np.zeros_like(atlas_data)

        for i, row in df.iterrows():
            # Get the value from the current atlas
            old_value = row['A424']
            # Get the new value from the df
            new_value = row[new_atlas]

            # Replace in new_atlas_data
            new_atlas_data[atlas_data == old_value] = new_value

        # Create a new NIfTI image
        new_atlas_img = nib.Nifti1Image(new_atlas_data, atlas_affine, atlas_header)
        # Save the new atlas
        new_atlas_path = Path(atlas).parent / f"{new_atlas.replace("-", "")}{suffix}.nii.gz"
        nib.save(new_atlas_img, new_atlas_path)
        print(f"Saved new atlas: {new_atlas_path}")

    print(f"Finished processing atlas: {atlas}\n")

Processing atlas: ./data/A424.nii.gz
Processing column: AA-7
Saved new atlas: data/AA7.nii.gz
Processing column: AA-24
Saved new atlas: data/AA24.nii.gz
Processing column: AA-50
Saved new atlas: data/AA50.nii.gz
Finished processing atlas: ./data/A424.nii.gz

Processing atlas: ./data/A424+2mm.nii.gz
Processing column: AA-7
Saved new atlas: data/AA7+2mm.nii.gz
Processing column: AA-24
Saved new atlas: data/AA24+2mm.nii.gz
Processing column: AA-50
Saved new atlas: data/AA50+2mm.nii.gz
Finished processing atlas: ./data/A424+2mm.nii.gz

Processing atlas: ./data/A424+4mm.nii.gz
Processing column: AA-7
Saved new atlas: data/AA7+4mm.nii.gz
Processing column: AA-24
Saved new atlas: data/AA24+4mm.nii.gz
Processing column: AA-50
Saved new atlas: data/AA50+4mm.nii.gz
Finished processing atlas: ./data/A424+4mm.nii.gz

Processing atlas: ./data/A424+n90merged.nii.gz
Processing column: AA-7
Saved new atlas: data/AA7+n90merged.nii.gz
Processing column: AA-24
Saved new atlas: data/AA24+n90merged.nii.gz


## Download the necessary files from templateflow
We will download the T1w in 1mm and  2mm voxel and the transformation files from templateflow.

In [6]:
files = [
    "https://templateflow.s3.amazonaws.com/tpl-MNI152NLin6Asym/tpl-MNI152NLin6Asym_res-01_desc-brain_T1w.nii.gz",
    "https://templateflow.s3.amazonaws.com/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz",
    "https://templateflow.s3.amazonaws.com/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_T1w.nii.gz",
    "https://templateflow.s3.amazonaws.com/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5",
]

for file in files:
    fname = file.split("/")[-1]
    print(f"Downloading {fname}...")
    with open(f"./data/{fname}", "wb") as f:
        response = requests.get(file)
        f.write(response.content)

## Transform the atlas into MNI152NLin2009cASym space

In [2]:
atlases = [
    "./data/AA7.nii.gz",
    "./data/AA7+2mm.nii.gz",
    "./data/AA7+4mm.nii.gz",
    "./data/AA7+n90merged.nii.gz",
    "./data/AA24.nii.gz",
    "./data/AA24+2mm.nii.gz",
    "./data/AA24+4mm.nii.gz",
    "./data/AA24+n90merged.nii.gz",
    "./data/AA50.nii.gz",
    "./data/AA50+2mm.nii.gz",
    "./data/AA50+4mm.nii.gz",
    "./data/AA50+n90merged.nii.gz",
    "./data/AA424.nii.gz",
    "./data/AA424+2mm.nii.gz",
    "./data/AA424+4mm.nii.gz",
    "./data/AA424+n90merged.nii.gz",
]

In [12]:
# data for sidecars
json_sidecar = {
    "Name": "A424",
    "Authors": [
        "Chadi G. Abdallah",
        "Teddy J. Akiki",
    ],
    "BIDSVersion": "1.10.0",
    "Space": "MNI152NLin2009cAsym"
}

df = pd.read_csv("./data/A424_Labels_AA-AAc_main_maps.csv",)
aa424_df = df[["AA-424","AA-424-Abbreviation","AA-424-Name"]]
aa424_df.columns = ["index", "label", "region_name"]

aa24_df = df[["AA-24","AA-24-Abbreviation","AA-24-Name"]]
aa24_df.columns = ["index", "label", "region_name"]
aa24_df = aa24_df.sort_values(by="index").drop_duplicates()

aa50_df = df[["AA-50","AA-50-Abbreviation","AA-50-Name"]]
aa50_df.columns = ["index", "label", "region_name"]
aa50_df = aa50_df.sort_values(by="index").drop_duplicates()

aa7_df = df[["AA-7","AA-7-Abbreviation","AA-7-Name"]]
aa7_df.columns = ["index", "label", "region_name"]
aa7_df = aa7_df.sort_values(by="index").drop_duplicates()

# create dictionary for sidecars
tsv_sidecars = {
    "AA424": aa424_df,
    "AA24": aa24_df,
    "AA50": aa50_df,
    "AA7": aa7_df
}

In [ ]:
transform = "./data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5"
resolutions = ["01", "02"]
mni2009 = "MNI152NLin2009cAsym"
mni6 = "MNI152NLin6Asym"

for resolution in resolutions:
    space = f"space-MNI152NLin2009cAsym_res-{resolution}"
    fixed = ants.image_read(f"./data/tpl-MNI152NLin2009cAsym_res-{resolution}_desc-brain_T1w.nii.gz")

    for atlas in atlases:
        atlas_name = atlas.split("/")[-1].replace(".nii.gz", "")
        out_path = Path(f"{atlas_name}")
        out_path.mkdir(parents=True, exist_ok=True)

        if "+" in atlas_name:
            atlas_name, desc_suffix = atlas_name.split("+")
            desc_suffix = f"_desc-{desc_suffix}"
        else:
            desc_suffix = ""

        mni2009_name = f"atlas-{atlas_name}_{space}{desc_suffix}_dseg"
        mni6_name = f"atlas-{atlas_name}_space-MNI152NLin6Asym_res-{resolution}{desc_suffix}_dseg"

        moving = ants.image_read(atlas)
        warped_image = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=transform, interpolator="genericLabel", verbose=True)
        warped_image.image_write(str(out_path / (mni2009_name + ".nii.gz"))) # In MNI152NLin2009cAsym space


        # Create sidecar JSON
        # for the atlas in MNI152NLin2009cAsym space
        json_sidecar["Resolution"] = f"{int(resolution)}mm"
        json_sidecar["Name"] = atlas_name
        json_sidecar["Description"] = f"{atlas_name} atlas in {mni2009} space"
        json_sidecar["Space"] = mni2009

        with open(out_path / f"{mni2009_name}.json", "w") as f:
            json.dump(json_sidecar, f, indent=4)

        # Create TSV file
        tsv_file = tsv_sidecars[atlas_name]
        tsv_file.to_csv(out_path / f"atlas-{atlas_name}_dseg.tsv", sep="\t", index=False)

        if resolution == "02": # only create files for 2mm resolution
            moving.image_write(str(out_path / (mni6_name + ".nii.gz"))) # Original atlas in MNI152NLin6Asym space

            # for the atlas in MNI152NLin6Asym space
            json_sidecar["Description"] = f"{atlas_name} atlas in {mni6} space"
            json_sidecar["Space"] = mni6

            with open(out_path / f"{mni6_name}.json", "w") as f:
                json.dump(json_sidecar, f, indent=4)

['-d', '3', '-i', '0x15d2aa888', '-o', '0x15d2aab68', '-r', '0x15d2aaac8', '-n', 'genericLabel', '-t', './data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5']
Using double precision for computations.
Input scalar image: 0x15d2aa888
Could not create ImageIO for the input file, assuming dimension = 3 and scalar pixel type
Reference image: 0x15d2aaac8
The composite transform comprises the following transforms (in order): 
  1. ./data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[0] (type = AffineTransform)
  2. ./data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[1] (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: 0x15d2aab68
['-d', '3', '-i', '0x15d2aaae8', '-o', '0x15d2aab68', '-r', '0x15d2aaa08', '-n', 'genericLabel', '-t', './data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5']
Using double precision for computations.
Input sc